<a href="https://colab.research.google.com/github/11aleksey11/connector_neo4j/blob/main/test_neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install py2neo
!pip install neo4j

*Получаем url для запроса к файлу *.csv и создаём dataframe*

In [ ]:
import pandas as pd
import requests
from py2neo import Graph
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
 

base_url = 'https://drive.google.com/file/d/1Q7r_rwyB91f1eiVDvjPfUsITzhxPV08M/view?usp=sharing'
url='https://drive.google.com/uc?id=' + base_url.split('/')[-2]
df = pd.read_csv(url)

lst_person = []
rows = ''
for i in df.iteritems():
  rows = i[1]
for row in rows:
  lst_person.append({'parent': row.split(';')[1], 'child': row.split(';')[2], 'relationship': row.split(';')[0]})


#создаём подключение к neo4j

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_friendship(self, lst):
        with self.driver.session(database="neo4j") as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.execute_write(
                self._create_and_return_friendship, lst)


    @staticmethod
    def _create_and_return_friendship(tx, lst):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        dct_person = {}
        counter = 1
        query = ''
        for dct in lst:
          if dct['child'] not in dct_person:
            person = 'person' + str(counter)
            query = ', '.join([f'({dct["child"].replace(" ", "_")}:Person {{ name: "{dct["child"]}"}})',  query])        
            dct_person[dct['child']] = person
            counter += 1
          if dct['parent'] not in dct_person.keys():
            person = 'person' + str(counter)
            query = ', '.join([f'({dct["parent"].replace(" ", "_")}:Person {{ name: "{dct["parent"]}"}})',  query])    
            dct_person[dct["parent"]] = person
            counter += 1
          #query =', '.join([query, f'({dct_person[dct["parent"]]})-[:id_event_{dct["relationship"]}]->({dct_person[dct["child"]]})'])
          query =', '.join([query, f'({dct["parent"].replace(" ", "_")})-[:EVENT {{name: "{dct["relationship"]}"}}]->({dct["child"].replace(" ", "_")})'])
        result = tx.run('create ' + query.replace(', ,', ', '))
        return [result]


if __name__ == "__main__":
    # Aura queries use an encrypted connection using the "neo4j+s" URI scheme
    uri = "neo4j+s://e2300488.databases.neo4j.io"
    user = "neo4j"
    password = "X-3JMVu5spbZqt-J_Ov88ceI8vbrdFVDLK9i4MrovB0"
    app = App(uri, user, password)
    app.create_friendship(lst_person)     
    app.close()